In [2]:
import os
import sys
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
import seaborn as sns
import json
import pathlib
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import pearsonr
import copy

In [87]:
def cvPearson(model, n_splits):
    def pearson_score(y_true, y_pred):
        return pearsonr(y_true, y_pred)[0]

    # Создание объекта KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Выполнение кросс-валидации
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        score = pearson_score(y_test, y_pred)
        scores.append(score)

    # Вывод среднего значения коэффициента Пирсона по всем фолдам
    return np.mean(scores)


In [ ]:
for model in [AbRFR, extra_trees_model, xgb_model, catboost_model]:
    mean_pearson = cvPearson(model, 5)
    print(f"Средний коэффициент Пирсона для {model.__class__.__name__}: {mean_pearson:.3f}")
    mean_r2 = np.mean(cross_val_score(model, X, y, cv=5, scoring='r2'))
    print(f"Средний R-squared для {model.__class__.__name__}: {mean_r2:.3f}")

Средний коэффициент Пирсона для RandomForestRegressor: 0.711
Средний R-squared для RandomForestRegressor: 0.463
Средний коэффициент Пирсона для ExtraTreesRegressor: 0.629
Средний R-squared для ExtraTreesRegressor: 0.282
Средний коэффициент Пирсона для XGBRegressor: 0.884
Средний R-squared для XGBRegressor: 0.948
Средний коэффициент Пирсона для CatBoostRegressor: 0.773
Средний R-squared для CatBoostRegressor: 0.615

In [98]:
feature_importances = AbRFR.feature_importances_
feature_names = X_train.columns

for name, importance in zip(feature_names, feature_importances):
    print(f"Важность признака {name}: {importance:.3f}")


Важность признака Hyd_Hyd_4: 0.006
Важность признака Hyd_Pos_4: 0.002
Важность признака Hyd_Neg_4: 0.002
Важность признака Hyd_Acc_4: 0.003
Важность признака Hyd_Don_4: 0.004
Важность признака Hyd_Aro_4: 0.004
Важность признака Hyd_Sul_4: 0.003
Важность признака Hyd_Neu_4: 0.006
Важность признака Pos_Hyd_4: 0.002
Важность признака Pos_Pos_4: 0.001
Важность признака Pos_Neg_4: 0.000
Важность признака Pos_Acc_4: 0.001
Важность признака Pos_Don_4: 0.001
Важность признака Pos_Aro_4: 0.002
Важность признака Pos_Sul_4: 0.000
Важность признака Pos_Neu_4: 0.002
Важность признака Neg_Hyd_4: 0.002
Важность признака Neg_Pos_4: 0.001
Важность признака Neg_Neg_4: 0.000
Важность признака Neg_Acc_4: 0.002
Важность признака Neg_Don_4: 0.002
Важность признака Neg_Aro_4: 0.001
Важность признака Neg_Sul_4: 0.000
Важность признака Neg_Neu_4: 0.002
Важность признака Acc_Hyd_4: 0.004
Важность признака Acc_Pos_4: 0.002
Важность признака Acc_Neg_4: 0.002
Важность признака Acc_Acc_4: 0.006
Важность признака Ac

In [117]:
abrfr_imp = AbRFR.feature_importances_
extra_trees_imp = extra_trees_model.feature_importances_
xgb_imp = xgb_model.feature_importances_
catboost_imp = catboost_model.get_feature_importance()

# Вывод средней важности каждого признака отдельно
for name, abrfr, extra_trees, xgb, catboost in zip(feature_names, abrfr_imp, extra_trees_imp, xgb_imp, catboost_imp):
    mean_imp = np.mean([abrfr, extra_trees, xgb])
    print(f"Признак {name}:")
    print(f"Средняя важность: {mean_imp:.3f}")
    print(f" AbRFR: {abrfr:.3f}")
    print(f" ExtraTreesRegressor: {extra_trees:.3f}")
    print(f" XGBRegressor: {xgb:.3f}")
    print(f" CatBoostRegressor: {catboost:.3f}")

Признак Hyd_Hyd_4:
Средняя важность: 0.004
 AbRFR: 0.006
 ExtraTreesRegressor: 0.006
 XGBRegressor: 0.001
 CatBoostRegressor: 0.194
Признак Hyd_Pos_4:
Средняя важность: 0.002
 AbRFR: 0.002
 ExtraTreesRegressor: 0.003
 XGBRegressor: 0.001
 CatBoostRegressor: 0.295
Признак Hyd_Neg_4:
Средняя важность: 0.002
 AbRFR: 0.002
 ExtraTreesRegressor: 0.002
 XGBRegressor: 0.003
 CatBoostRegressor: 0.276
Признак Hyd_Acc_4:
Средняя важность: 0.002
 AbRFR: 0.003
 ExtraTreesRegressor: 0.002
 XGBRegressor: 0.002
 CatBoostRegressor: 0.067
Признак Hyd_Don_4:
Средняя важность: 0.004
 AbRFR: 0.004
 ExtraTreesRegressor: 0.004
 XGBRegressor: 0.006
 CatBoostRegressor: 0.584
Признак Hyd_Aro_4:
Средняя важность: 0.005
 AbRFR: 0.004
 ExtraTreesRegressor: 0.004
 XGBRegressor: 0.008
 CatBoostRegressor: 0.332
Признак Hyd_Sul_4:
Средняя важность: 0.003
 AbRFR: 0.003
 ExtraTreesRegressor: 0.004
 XGBRegressor: 0.001
 CatBoostRegressor: 0.263
Признак Hyd_Neu_4:
Средняя важность: 0.004
 AbRFR: 0.006
 ExtraTreesRegresso

### То же для фичей Clark et al.

In [118]:
# Загружаю датасет от Clark et al.
features = ["idSASA_fraction_1", "idhSASA_sc_0", "idSASA_0", "dE2", "iinterface_residues_1", "idSASA_fraction_0", "idSASA_sc_0", "fa_rep_1", "hbond_bb_sc_0", "fa_atr_1", "iinterface_hbonds", "isc_value", "idelta_unsat_hbonds", "fa_elec_0", "sin_res", "sin_norm", "lk_ball_wtd_1", "fa_atr_0", "total_score_0", "fa_sol_0", "hbond_bb_sc_1", "total_score_1", "hbond_sc_0", "idhSASA_sc_1", "fa_elec_1", "idG_1", "fa_sol_1", "aif_score", "sin_if", "idhSASA_1", "fa_rep_0"]

X = pd.read_csv('./AbRFC/data/train/Xtrain.csv')
y = X['ddG']
y_clf = X['y_clf']
X = X.loc[:, features]
X['aif_score'] =np.where(X['aif_score']=='False',np.NaN,X['aif_score']) #non-Ab/Ag PPI have no AIF score
X = X.drop('aif_score', axis=1) #!!! пока не придумал, что делать с NaN


X_train, X_test, y_train_clf, y_test_clf = train_test_split(X, y_clf, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [119]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
X

,idSASA_fraction_1,idhSASA_sc_0,idSASA_0,dE2,iinterface_residues_1,idSASA_fraction_0,idSASA_sc_0,fa_rep_1,hbond_bb_sc_0,fa_atr_1,iinterface_hbonds,isc_value,idelta_unsat_hbonds,fa_elec_0,sin_res,sin_norm,lk_ball_wtd_1,fa_atr_0,total_score_0,fa_sol_0,hbond_bb_sc_1,total_score_1,hbond_sc_0,idhSASA_sc_1,fa_elec_1,idG_1,fa_sol_1,sin_if,idhSASA_1,fa_rep_0
0,0.000629,-1.539380,8.861673,-1.065199,0.0,0.095146,8.861673,0.000000,0.000000,0.000000,0.0,-0.000131,0.0,-7.654828,2.692408,1.061321,0.000000e+00,-2.372275,-1.399372,6.988184,0.000000,0.000000,0.000000,0.000000,0.000000,-0.128561,0.000000,-11.941427,0.000000,2.362774
1,0.006397,0.000000,0.512343,-2.944844,0.0,0.003722,0.512343,0.203296,0.000000,-0.062101,0.0,0.000000,0.0,0.001664,0.334530,0.165094,-1.391109e-01,-3.294486,-2.344316,0.526314,0.000000,1.098722,0.000000,0.000000,1.511687,0.000842,-0.323566,-2.257936,0.000000,0.972294
2,0.017849,0.000000,0.000000,-1.022368,0.0,-0.031816,0.000000,0.000000,-1.059560,-0.004397,0.0,-0.000871,-2.0,-1.113327,0.563188,0.306604,-1.110223e-16,-2.689247,-0.614268,4.092781,0.000000,-0.278757,0.000000,0.000000,-0.274360,0.433646,0.000000,-1.696279,0.000000,0.310906
3,0.523725,-5.387831,6.071217,1.056705,0.0,0.001400,6.071217,0.000000,0.000000,0.000000,0.0,-0.003818,3.0,-0.417241,0.925037,0.306604,0.000000e+00,-3.592204,0.682902,3.136397,0.000000,0.000000,0.000000,-1.539380,0.000000,0.576937,0.000000,-3.176209,-0.055920,3.235450
4,0.012932,-1.539380,-2.788990,-0.998932,0.0,-0.190880,-2.788990,0.000000,-0.561960,0.000000,0.0,-0.001187,0.0,-1.556793,1.472897,0.471698,0.000000e+00,-4.279654,-1.740407,3.420474,0.000000,0.000000,0.000000,0.000000,0.000000,-0.157532,0.000000,-4.862233,0.000000,2.542368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,-0.254232,-5.387831,-17.171728,0.322603,0.0,0.025742,-19.733445,0.000000,0.000000,0.000000,0.0,-0.007616,-2.0,0.506956,-3.048746,-0.340909,0.000000e+00,2.482489,0.887977,-2.617794,0.000000,0.000000,0.531885,-1.539380,0.000000,-0.600608,0.000000,-1.127172,-1.883404,-0.024046
1796,-5.152688,-0.769690,-18.778385,2.487863,-6.0,-1.000000,-18.778385,-5.499716,0.687802,21.277547,-1.0,0.009462,-4.0,1.054173,-6.824126,-0.454545,-1.842029e-01,12.435682,5.590799,-5.489516,3.152903,24.055924,0.000000,-115.453530,8.518457,13.340188,-7.198159,-4.060843,-197.185305,-6.354454
1797,-3.549227,-1.539380,-32.008951,-3.733916,-5.0,-0.384813,-32.008951,-5.263779,-0.073935,23.239959,0.0,0.007542,1.0,0.125768,-4.414065,-0.416667,5.353973e-01,8.442643,-1.019288,-3.840990,2.586064,1.102592,0.338348,-33.866369,0.254724,1.860716,-23.123265,-1.641822,-93.548384,-11.507350
1798,-0.841710,-40.023890,-72.590765,0.294269,-1.0,0.087489,-72.143960,-5.359920,-0.124179,11.227294,0.0,-0.011658,-2.0,0.495308,-8.704791,-0.492424,-9.701238e-01,9.520398,3.831010,-3.686048,0.000000,7.500463,0.000000,6.927212,6.924868,-0.507271,-6.718467,-6.860318,6.927212,-4.286926


In [120]:
# Создание и обучение модели RandomForestClassifier
AbRFC = RandomForestClassifier(**clf_params)
AbRFC.fit(X_train, y_train_clf)

# Вычисление коэффициента Пирсона для RandomForestClassifier
y_pred_clf = AbRFC.predict(X_test)
correlation_coefficient_clf, p_value = pearsonr(y_pred_clf, y_test_clf)
print("Коэффициент Пирсона:", correlation_coefficient_clf)

Коэффициент Пирсона: 0.5519760527185088


In [121]:
# Создание и обучение модели RandomForestRegressor
AbRFR = RandomForestRegressor(**reg_params)
AbRFR.fit(X_train, y_train)

# Вычисление коэффициента Пирсона для RandomForestRegressor
y_pred = AbRFR.predict(X_test)
correlation_coefficient, p_value = pearsonr(y_pred, y_test)
print("Коэффициент Пирсона RandomForestRegressor:", correlation_coefficient)

Коэффициент Пирсона RandomForestRegressor: 0.7064387111034802


In [122]:
# Создание и обучение модели ExtraTreesRegressor
extra_trees_model = ExtraTreesRegressor(**reg_params)
extra_trees_model.fit(X_train, y_train)

# Вычисление коэффициента Пирсона
y_pred_extra_trees = extra_trees_model.predict(X_test)
correlation_extra_trees, p_value_extra_trees = pearsonr(y_test, y_pred_extra_trees)
print("Коэффициент Пирсона ExtraTreesRegressor:", correlation_extra_trees)

Коэффициент Пирсона ExtraTreesRegressor: 0.6379447258026569


In [123]:
# Создание и обучение модели XGBRegressor
xgb_model = XGBRegressor(n_estimators=1000, max_depth=6, learning_rate=0.01)
xgb_model.fit(X_train, y_train)

# Вычисление коэффициента Пирсона для XGBRegressor
y_pred_xgb = xgb_model.predict(X_test)
correlation_xgb, p_value_xgb = pearsonr(y_test, y_pred_xgb)
print("Коэффициент Пирсона XGBRegressor:", correlation_xgb)

Коэффициент Пирсона XGBRegressor: 0.7219555178265762


In [125]:
# Создание и обучение модели CatBoostRegressor
catboost_model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.01, loss_function='RMSE')
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

# Вычисление коэффициента Пирсона для CatBoost
y_pred_catboost = catboost_model.predict(X_test)
correlation_catboost, p_value_catboost = pearsonr(y_test, y_pred_catboost)
print("Коэффициент Пирсона CatBoostRegressor:", correlation_catboost)

Коэффициент Пирсона CatBoostRegressor: 0.7213385676059049


In [126]:
cross_val_score(AbRFR, X, y, cv=5, scoring='r2')

array([0.07924862, 0.0422503 , 0.49380346, 0.06997966, 0.03968901])

In [127]:
def cvPearson(model, n_splits):
    def pearson_score(y_true, y_pred):
        return pearsonr(y_true, y_pred)[0]

    # Создание объекта KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Выполнение кросс-валидации
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        score = pearson_score(y_test, y_pred)
        scores.append(score)

    # Вывод среднего значения коэффициента Пирсона по всем фолдам
    return np.mean(scores)


In [ ]:
for model in [AbRFR, extra_trees_model, xgb_model, catboost_model]:
    mean_pearson = cvPearson(model, 5)
    print(f"Средний коэффициент Пирсона для {model.__class__.__name__}: {mean_pearson:.3f}")
    mean_r2 = np.mean(cross_val_score(model, X, y, cv=5, scoring='r2'))
    print(f"Средний R-squared для {model.__class__.__name__}: {mean_r2:.3f}")

Средний коэффициент Пирсона для RandomForestRegressor: 0.672
Средний R-squared для RandomForestRegressor: 0.143
Средний коэффициент Пирсона для ExtraTreesRegressor: 0.601
Средний R-squared для ExtraTreesRegressor: -0.018
Средний коэффициент Пирсона для XGBRegressor: 0.694
Средний R-squared для XGBRegressor: 0.135
Средний коэффициент Пирсона для CatBoostRegressor: 0.689
Средний R-squared для CatBoostRegressor: 0.180

In [134]:
abrfr_imp = AbRFR.feature_importances_
extra_trees_imp = extra_trees_model.feature_importances_
xgb_imp = xgb_model.feature_importances_
catboost_imp = catboost_model.get_feature_importance()

feature_names = X_train.columns

# Вывод средней важности каждого признака отдельно
for name, abrfr, extra_trees, xgb, catboost in zip(feature_names, abrfr_imp, extra_trees_imp, xgb_imp, catboost_imp):
    mean_imp = np.mean([abrfr, extra_trees, xgb])
    print(f"Признак {name}:")
    print(f"Средняя важность: {mean_imp:.3f}")
    print(f"    AbRFR: {abrfr:.3f}")
    print(f"    ExtraTreesRegressor: {extra_trees:.3f}")
    print(f"    XGBRegressor: {xgb:.3f}")
    print(f"    CatBoostRegressor: {catboost:.3f}")
    print()

Признак idSASA_fraction_1:
Средняя важность: 0.056
    AbRFR: 0.071
    ExtraTreesRegressor: 0.082
    XGBRegressor: 0.015
    CatBoostRegressor: 3.427

Признак idhSASA_sc_0:
Средняя важность: 0.012
    AbRFR: 0.011
    ExtraTreesRegressor: 0.013
    XGBRegressor: 0.013
    CatBoostRegressor: 3.754

Признак idSASA_0:
Средняя важность: 0.035
    AbRFR: 0.039
    ExtraTreesRegressor: 0.047
    XGBRegressor: 0.018
    CatBoostRegressor: 3.013

Признак dE2:
Средняя важность: 0.031
    AbRFR: 0.052
    ExtraTreesRegressor: 0.005
    XGBRegressor: 0.036
    CatBoostRegressor: 5.359

Признак iinterface_residues_1:
Средняя важность: 0.043
    AbRFR: 0.031
    ExtraTreesRegressor: 0.085
    XGBRegressor: 0.012
    CatBoostRegressor: 1.677

Признак idSASA_fraction_0:
Средняя важность: 0.025
    AbRFR: 0.022
    ExtraTreesRegressor: 0.031
    XGBRegressor: 0.020
    CatBoostRegressor: 4.065

Признак idSASA_sc_0:
Средняя важность: 0.033
    AbRFR: 0.028
    ExtraTreesRegressor: 0.039
    XGBRegres

In [ ]:
def feature_importances(rf,features_keep,fold):
     ### Calculate the feature importances given a random forest classifier
     ##  values for all trees in the forest are used as to estimate the distribution.
     ##  graph is ranked based on median and may fluctuate slightly from run to run.
     fimps = []
     std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
     for k in range(len(rf.feature_importances_)):
          fimps+=[{'feature':features_keep[k],'importance':np.round(rf.feature_importances_[k],3),
                              'importance_std':np.round(std[k],3), 'fold':fold}]
     return fimps

In [ ]:
FILE_PATH = os.getcwd() + '/'

def run_cv():
    #runs the cross-validation for AbRFC, RF Regressor
    #saves a csv with all the predicitions for all models and folds
    DATA_DIR = FILE_PATH+'./AbRFC/data/train/'
    CV = json.load(open(DATA_DIR+'CVIDS.json','r'))
    
    all_fimps = [] #to store feature importances 
    all_data = []  #to store model predictions

    for FOLD in range(5):
        ix_train, ix_test = CV[str(FOLD)]     

        train_groups = X.loc[ix_train,'pdb_group'].unique()
        test_groups  = X.loc[ix_test,'pdb_group'].unique()
        assert len(set(train_groups).intersection(set(test_groups)))==0

        ######GET LABELS############
        ytrain_reg  = X.loc[ix_train,'ddG'].values
        ytest_reg = X.loc[ix_test,'ddG'].values
        ############################

        #####AbRFC######
        xtrain_tree = X.loc[ix_train,features].values
        
        #feature importances
        all_fimps+=feature_importances(AbRFR, features,FOLD)

        #####RF REGRESSOR####
        reg = fitAbRFR(xtrain_tree,ytrain_reg,REG_PARAMS)
        #predictions
        ypred_reg = applyAbRFR(X.loc[ix_test,params['features']].values,reg)
        ####REGRESSION END####

        for i in range(len(ix_test)):
            d = {'FOLD':FOLD,'ID':ix_test[i],'y_reg':ytest_reg[i], 
                        'y_clf':ytest[i],'AbRFC':ypred[i],'RF Regressor':ypred_reg[i],
                        'GNN Classifier':ypred_gnn[i],'refAA':X.loc[ix_test[i],'refAA'],
                        'mutAA':X.loc[ix_test[i],'mutAA']}
            if ix_test[i] in ix_test_abl:
                ix_abl = ix_test_abl.index(ix_test[i])
                d['AbLang'] = ypred_abl[ix_abl]
            all_data+=[d]

    all_fimps = pd.DataFrame.from_records(all_fimps)
    print(f'Writing Feature Importances to {feature_importances.csv}')
    all_fimps.to_csv(feature_importances.csv,index=False)
    print(f'Writing all data to {cv_preds.csv}')
    pd.DataFrame.from_records(all_data).to_csv('./cv_preds.csv',index=False)